In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import taxamodelinghelper as helper

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.dummy import DummyClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
# from adspy_shared_utilities import plot_feature_importances
# #from adspy_shared_utilities import plot_decision_tree

import joblib
import os
from datetime import datetime

# supress future warnings
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [2]:
# read in data
p = '../data/'
input_name = 'cleaned.csv'

df = pd.read_csv(f'{p}{input_name}')

In [3]:
df['ship_date_mm'] = df['ship_date_mm'].astype(str)
df.head()

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
0,2018280858,WDER,ODOCOILEUS,VIRGINIANUS,MEA,MAM,NaN_cartons,1.0,NO,500.0,...,LR,WHITE-TAILED DEER,True,False,False,False,False,1.0,2018,6
1,2018301780,BLBE,URSUS,AMERICANUS,TRO,MAM,NaN_cartons,1.0,NO,0.0,...,DE,AMERICAN BLACK BEAR,True,False,False,False,False,1.0,2018,6
2,2018311221,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,2.0,NO,0.0,...,NG,BALD EAGLE,True,False,False,False,False,2.0,2018,6
3,2018311221,FEHA,BUTEO,REGALIS,FEA,RAP,NaN_cartons,2.0,NO,0.0,...,NG,FERRUGINOUS HAWK,True,False,False,False,False,2.0,2018,6
4,2018311221,BAOW,TYTO,ALBA,FEA,RAP,NaN_cartons,1.0,NO,0.0,...,NG,BARN OWL,True,False,False,False,False,1.0,2018,6


### Bald Eagles

In [4]:
df[df.genus == 'HALIAEETUS'].sample(5)  #547 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
850718,2018449603,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,9.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,9.0000,2018,8
1079569,2017106519,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,4.0,NO,0.0,...,6,BALD EAGLE,True,False,False,False,False,4.0000,2016,6
1471360,2018287268,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,30.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,30.0000,2017,8
1080542,2017164864,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,2.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,2.0000,2016,5
1884991,2016868750,HALC,HALIAEETUS,LEUCOCEPHALUS,SPE,EGL,1.0,28.5,ML,0.0,...,ME,BALD EAGLE,False,False,True,False,False,0.0285,2016,5


In [5]:
df[df.genus == 'HALIAEETUS'].specific_generic_name.unique()

array(['BALD EAGLE', 'WHITE-TAILED SEA EAGLE', 'AFRICAN FISH EAGLE',
       "STELLER'S SEA EAGLE"], dtype=object)

In [18]:
df[df.genus == 'HALIAEETUS'].species_code.unique() #31985 rows

array(['BAEA', 'HALC', 'HALP', 'HAAL', 'HAVO', 'SSEA'], dtype=object)

In [6]:
df[df.species_code == 'BAEA'].sample(5) #475 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
335610,2017106170,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,11.0,NO,0.0,...,6,BALD EAGLE,True,False,False,False,False,11.0,2017,2
1471193,2018285876,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,24.0,NO,0.0,...,PL,BALD EAGLE,True,False,False,False,False,24.0,2017,7
850717,2018449601,BAEA,HALIAEETUS,LEUCOCEPHALUS,BON,EGL,NaN_cartons,2.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,2.0,2018,9
318598,2019648407,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,1.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,1.0,2019,9
1682342,2017176329,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,20.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,20.0,2016,7


In [7]:
df[df.specific_generic_name == 'BALD EAGLE'].sample(5) #537 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
1076460,2016912426,BAEA,HALIAEETUS,LEUCOCEPHALUS,WNG,EGL,NaN_cartons,1.0,NO,0.0,...,SW,BALD EAGLE,True,False,False,False,False,1.0,2016,4
1682430,2017176517,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,6.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,6.0,2016,9
107538,2019550481,BAEA,HALIAEETUS,LEUCOCEPHALUS,TRO,EGL,NaN_cartons,6.0,NO,0.0,...,6,BALD EAGLE,True,False,False,False,False,6.0,2018,6
849832,2018441048,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,4.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,4.0,2018,7
1470799,2018281379,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,38.0,NO,0.0,...,DS,BALD EAGLE,True,False,False,False,False,38.0,2017,7


In [8]:
df[df['act']=='R']

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
0,2018280858,WDER,ODOCOILEUS,VIRGINIANUS,MEA,MAM,NaN_cartons,1.0,NO,500.0,...,LR,WHITE-TAILED DEER,True,False,False,False,False,1.0,2018,6
2,2018311221,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,2.0,NO,0.0,...,NG,BALD EAGLE,True,False,False,False,False,2.0,2018,6
3,2018311221,FEHA,BUTEO,REGALIS,FEA,RAP,NaN_cartons,2.0,NO,0.0,...,NG,FERRUGINOUS HAWK,True,False,False,False,False,2.0,2018,6
4,2018311221,BAOW,TYTO,ALBA,FEA,RAP,NaN_cartons,1.0,NO,0.0,...,NG,BARN OWL,True,False,False,False,False,1.0,2018,6
5,2018311221,BAEA,HALIAEETUS,LEUCOCEPHALUS,FEA,EGL,NaN_cartons,1.0,NO,0.0,...,NG,BALD EAGLE,True,False,False,False,False,1.0,2018,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890058,2017165688,STCA,STRUTHIO,CAMELUS,FEA,OBR,1.0,294.0,MT,1775.0,...,ME,SOUTHERN AFRICAN OSTRICH,False,False,False,True,False,294.0,2017,5
1890100,2020738943,LOL?,LOLIGO,SPECIES,MEA,MOL,NaN_cartons,48.0,MT,188400.0,...,SF,SQUID,False,False,False,True,False,48.0,2020,5
1890101,2020738944,LOL?,LOLIGO,SPECIES,MEA,MOL,NaN_cartons,48.0,MT,182400.0,...,SF,SQUID,False,False,False,True,False,48.0,2020,5
1890110,2016964160,CHIN,CHINCHILLA,LANIGERA,CLO,MAM,1.0,4.0,MT,1976.0,...,ME,LONG-TAILED CHINCHILLA,False,False,False,True,False,4.0,2016,11


### Python

In [9]:
df[df.genus == 'PYTHON'].species_code.unique() #31985 rows

array(['PYTM', 'PYTR', 'PYTB', 'PBRT', 'PBRO', 'PYTC', 'PYMO', 'PYT?',
       'PYTI', 'PYMM', 'PYSA', 'PYTS', 'PYAN'], dtype=object)

In [10]:
df[df.specific_generic_name == 'PYTHON'].sample(5)  #259 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
924140,2017161577,PYT?,PYTHON,SPECIES,LPL,REP,1.0,1.0,NO,680.0,...,AN,PYTHON,True,False,False,False,False,1.0,2017,5
391475,2017132059,MORN,MORELIA,NAUTA,LIV,REP,9.0,1.0,NO,125.0,...,MI,PYTHON,True,False,False,False,False,1.0,2017,2
1076650,2016913390,PYT?,PYTHON,SPECIES,LPS,REP,NaN_cartons,2.0,NO,50.0,...,NY,PYTHON,True,False,False,False,False,2.0,2016,4
1217268,2020740251,PYT?,PYTHON,SPECIES,SKP,REP,1.0,44.0,NO,40.0,...,LO,PYTHON,True,False,False,False,False,44.0,2020,5
1494455,2019488853,PYT?,PYTHON,SPECIES,LPS,REP,NaN_cartons,2.0,NO,40.0,...,EL,PYTHON,True,False,False,False,False,2.0,2019,1


### American Crocodiles

In [11]:
df[df.genus == 'ALLIGATOR'].sample(5) #69,846 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
560777,2020695610,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,2.0,NO,8537.0,...,NW,AMERICAN ALLIGATOR,True,False,False,False,False,2.0,2020,1
119349,2019530514,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,2.0,7.0,NO,3091.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,7.0,2019,4
1408447,2017201028,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,4.0,3.0,NO,33722.0,...,LA,AMERICAN ALLIGATOR,True,False,False,False,False,3.0,2017,8
328996,2017103280,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,1.0,NO,209950.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,1.0,2017,2
846190,2018426379,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,45.0,NO,3133.0,...,NW,AMERICAN ALLIGATOR,True,False,False,False,False,45.0,2018,9


In [17]:
df[df.genus == 'ALLIGATOR'].species_code.unique()

array(['ALLM'], dtype=object)

In [12]:
df[df.species_code == 'ALLM']

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
134,2018327962,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,10.0,298.0,NO,16675.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,298.0000,2018,4
146,2018328260,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,10.0,362.0,NO,22006.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,362.0000,2018,4
158,2018328391,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,1.0,NO,212.0,...,DF,AMERICAN ALLIGATOR,True,False,False,False,False,1.0000,2018,4
210,2018328646,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,1.0,11.0,NO,755.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,11.0000,2018,4
237,2018328840,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,74.0,168.0,NO,29860.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,168.0000,2018,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867587,2020685424,ALLM,ALLIGATOR,MISSISSIPPIENSIS,DER,REP,110.0,82000.0,GM,1320.0,...,PL,AMERICAN ALLIGATOR,False,True,False,False,False,82.0000,2020,1
1867591,2020686154,ALLM,ALLIGATOR,MISSISSIPPIENSIS,DER,REP,104.0,49000.0,GM,792.0,...,DE,AMERICAN ALLIGATOR,False,True,False,False,False,49.0000,2020,1
1886963,2016966494,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,1.0,1.0,ML,1.0,...,NW,AMERICAN ALLIGATOR,False,False,True,False,False,0.0010,2016,12
1888665,2019521180,ALLM,ALLIGATOR,MISSISSIPPIENSIS,SPE,REP,1.0,163.7,ML,1.0,...,MI,AMERICAN ALLIGATOR,False,False,True,False,False,0.1637,2019,3


In [13]:
df[df.specific_generic_name == 'AMERICAN ALLIGATOR'].sample(5) #69,846 rows

,control_number,species_code,genus,species,wildlf_desc,wildlf_cat,cartons,qty,unit,value,...,pt_cd,specific_generic_name,unit_NO,unit_KG,unit_LT,unit_MT,unit_M2,qty_new,ship_date_yyyy,ship_date_mm
904161,2017152849,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,3.0,4.0,NO,1678.0,...,DF,AMERICAN ALLIGATOR,True,False,False,False,False,4.0,2017,5
1355526,2017980276,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,1.0,NO,35254.0,...,NY,AMERICAN ALLIGATOR,True,False,False,False,False,1.0,2016,12
649493,2018435695,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,2.0,NO,404.0,...,MI,AMERICAN ALLIGATOR,True,False,False,False,False,2.0,2018,10
869048,2017136628,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,3.0,1.0,NO,6337.0,...,NW,AMERICAN ALLIGATOR,True,False,False,False,False,1.0,2017,4
652836,2018437414,ALLM,ALLIGATOR,MISSISSIPPIENSIS,LPS,REP,1.0,1.0,NO,297.0,...,HA,AMERICAN ALLIGATOR,True,False,False,False,False,1.0,2018,10


In [14]:
len(df.specific_generic_name.unique())

10747

In [15]:
df.groupby('specific_generic_name').size().sort_values(ascending=False)[:20]

specific_generic_name
GOLD-LIP OYSTER                101309
AMERICAN ALLIGATOR              69846
AMERICAN MINK                   40988
CORAL                           36057
AMERICAN BLACK BEAR             34535
STAGHORN CORAL                  32912
COMMERCIAL TOP SHELL            32628
SOUTHERN AFRICAN OSTRICH        28416
SHELLS MOLLUSCS                 27663
GREAT PEARL OYSTER              24160
MASKED WATER SNAKE              22141
ARCTIC FOX                      21975
CRABS,LOBSTERS, CRUSTACEANS     21967
ELK                             21473
ABALONE SHELL                   21094
WHITE-TAILED DEER               19162
SEA URCHIN                      18557
RETICULATED PYTHON              17556
RED FOX                         17006
IMPALA                          15969
dtype: int64

In [16]:
df.groupby('genus').size().sort_values(ascending=False)[:20]

genus
PINCTADA              137068
ALLIGATOR              69846
TROPICAL FISH          69252
NaN_genus              54953
NONCITES ENTRY         44440
NEOVISON               42431
VULPES                 40380
URSUS                  36041
TROCHUS                35396
ACROPORA               33805
PYTHON                 31985
HALIOTIS               29891
STRUTHIO               28433
OCTOPUS                25709
TRAGELAPHUS            24184
ODOCOILEUS             23508
OTHER LIVE INVERTS     22986
CERVUS                 22788
HOMALOPSIS             22141
CROCODYLUS             18342
dtype: int64